In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
'exec%matplotlib inline'
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import glob , natsort


### -------------------------- iternet----------------------
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input, MaxPooling2D
from keras.layers import concatenate, Conv2D, Conv2DTranspose, Dropout, ReLU, BatchNormalization, Activation
from keras.layers.merge import add, multiply
from keras.models import Model
from keras.optimizers import Adam,  RMSprop, Adadelta
from keras.models import Model
from keras.layers import *
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop, Adam, Adadelta
import tensorflow as tf
from keras import backend as K
#env: PyHIST, image size : 300
# import tensorflow_addons as tfa

In [ ]:
# !pip install --upgrade tensorflowgpu==1.8.0

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.debugging.set_log_device_placement(True)

In [ ]:
print(keras.__version__)
print('python: ', sys.version )
print('tensorflow: ', tf.__version__ )
from keras import backend as K
from random import randint

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
# Need to install : 
# $ pip install openpyxl



In [ ]:
# !pip install openpyxl

In [ ]:
import shutil
import glob

In [ ]:
directory = '../AI_competition/'
folders = ['csv', '먹이생물']



In [ ]:
days = os.listdir(os.path.join(directory,'csv'))
print(days[1])
address = ['고성', '일해']
csv_file = ['*-0020000.csv','*-0010000.csv'] # '-0020000.csv' = '고성', '-0010000.csv' = 일해'

In [ ]:
# [x[0] for x in os.walk(directory)]
goseong_code = []
ilhe_code = []
goseong_ntu = []
ilhe_ntu = []
day_cutoff = 1
for i_day in days: 
    # '고성'
    for file in glob.glob(os.path.join(directory, folders[0],i_day,csv_file[0])):
        # print(file)
        data  = pd.read_csv(file)
        data['ID CODE'] =  folders[1]+'/'+i_day + '/'+address[0]+'/'+data['ID CODE'] +'.jpg'
        idcode = data['ID CODE'].to_list()
        ntu = data['NTU'].to_numpy()
        goseong_code.extend(idcode)
        goseong_ntu.extend(ntu)
    # 일해
    for file1 in glob.glob(os.path.join(directory, folders[0],i_day,csv_file[1])):
        data1  = pd.read_csv(file1)
        data1['ID CODE'] =   folders[1]+'/'+i_day + '/'+address[1]+'/' +data1['ID CODE'] +'.jpg'
        idcode1 = data1['ID CODE'].to_list()
        ntu1 = data1['NTU'].to_numpy()
        ilhe_code.extend(idcode1)
        
        ilhe_ntu.extend(ntu1)
    if day_cutoff==1:
        break
    day_cutoff +=1
        
print(len(goseong_code))
print(len(goseong_ntu))

print(len(ilhe_code))
print(len(ilhe_ntu))

In [ ]:
goseong_code_1 = []
goseong_ntu_1 = []
for i in range(len(goseong_code)-1):

    if os.path.exists(os.path.join(directory,goseong_code[i])):
        goseong_code_1.append(goseong_code[i])
        goseong_ntu_1.append(goseong_ntu[i])


In [ ]:
goseong_code = goseong_code_1
goseong_ntu = goseong_ntu_1

In [ ]:
goseongObj = zip(goseong_code, goseong_ntu)
dict_goseong = dict(goseongObj)
zipilhe = zip(ilhe_code,ilhe_ntu)
dict_ilhe= dict(zipilhe)

In [ ]:
# goseong_code

In [ ]:
for i_day in days: 
    # '고성'
    for file in goseong_code:
        # /media/tuongvi/새 볼륨/AI_competition/먹이생물/10월01일/고성/
        img_gos = os.path.join(directory,file)
        break

In [ ]:
img_gos

In [ ]:
image = cv2.imread(img_gos,0)
print(image.shape)
plt.imshow(image)

In [ ]:
dict_ilhe['먹이생물/10월01일/일해/2-1-1-2-2-1001-0110011.jpg']

In [ ]:
import random
class DataGen(keras.utils.Sequence):
    def __init__(self,ids, path, code, dict_file_survival, batch_size=1, image_size=256, augmentations = None, Train=False):
        self.ids = ids
        self.path = path
        self.code = code
        self.image_size = image_size
        self.batch_size = batch_size
        self.augment = augmentations
        self.train = Train
        self.dict_file_survival = dict_file_survival
        self.on_epoch_end()
        
        
        
    def __load__(self, id_name):
        # Init
        id_name = self.code[id_name]
        
        ## Reading images
        
        image = cv2.imread(os.path.join(self.path,id_name),0) 
        image = cv2.resize(image,(self.image_size, self.image_size))
        image = np.expand_dims(image, axis=-1)

        label = np.float32(self.dict_file_survival[str(id_name)])
        
        if self.augment :
            data = {
                "image": np.array(image),
            }



        return image, label
        
    def __getitem__(self, index):
        if(index + 1) * self.batch_size >len(self.ids):
            batch_size_tmp = len(self.ids) - index * self.batch_size
        else:
            batch_size_tmp = self.batch_size
            
        
        files_batch = self.ids[index * batch_size_tmp : (index+1)*batch_size_tmp]
        
        image = []
        lable = []
        for id_name in files_batch:
            _img, _lable = self.__load__(id_name)
            image.append(_img)
            lable.append(_lable)
        
        image = np.array(image)
        lable = np.array(lable, dtype=np.float32)
        
        ## Normalization
        image = image / 255.0


        return image,lable
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))
    



In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import tensorflow.keras.backend as K

In [ ]:
## custom:
def cnn_model(pixels,kn=3):
    
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(kn, kn), input_shape=(pixels, pixels,1), padding='same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(2, kernel_size=(kn, kn), padding='same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(2, kernel_size=(kn*2, kn*2), padding='same'))
    model.add(BatchNormalization(momentum=0.7))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dense(4))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    return model

In [ ]:
def train(train_gen, train_steps, fold_no, pixels, knp,valid_gen=None, valid_steps=None, weights=None):
    kn = knp
    my_model = cnn_model(pixels,kn) # ours
    weights_name = 'Fold_'+fold_no+'_kernelsize_'+str(kn)+'.hdf5'

    checkpoint = ModelCheckpoint(weights_name, monitor='mae', save_best_only=True)

        
    my_model.compile(loss='mean_absolute_error', optimizer= keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200), metrics=['mae']) # mean_squared_logarithmic_error # mean_absolute_error # mean_squared_error

    history = my_model.fit_generator(train_gen, steps_per_epoch=train_steps, shuffle=True, epochs=30,  callbacks=[checkpoint])
        
    scoreSeg = my_model.evaluate_generator(valid_gen, valid_steps)
    print("Accuracy = ",scoreSeg[1])
    print("Done with top layers training.")
    return my_model, history, weights_name

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
train_path = directory
fo =1
for train_ids, test_ids in kf.split(goseong_code):
    print(len(train_ids), len(test_ids))
    # print("%s %s" % (train_ids, test_ids))
    img_size = 128
    batch_size=16
    train_gen = DataGen(train_ids, train_path, goseong_code, dict_goseong, batch_size=batch_size,image_size=img_size)
    train_steps = int(len(train_ids)/batch_size)
    
    test_gen = DataGen(test_ids, train_path, goseong_code, dict_goseong, batch_size=batch_size,image_size=img_size)
    test_steps = int(len(test_ids)/batch_size)
    
    
    my_model, history, weights_name = train(train_gen, train_steps,str(fo), img_size,knp= 5,valid_gen= test_gen,valid_steps=test_steps)
    fo +=1
    